### 1. Module Import

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0, 1' #2, 3'

from efficientnet_pytorch import EfficientNet  # --> Efficient Net pytorch library

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as dataset
import torchvision.transforms as transforms

from torch.utils.data import TensorDataset, SubsetRandomSampler

### 2. DATASET
- 2-1. Classic Dataset using torch
- 2-2. Custom Dataset using dataset template
- 2-3. ImageFolder using torchvision

In [ ]:
# 2-1. Classic Dataset using torch
# >>> torch.utils.data.TensorDataset

dataset = TensorDataset(inps, tgts)

In [ ]:
# 2-2. Custom Dataset using dataset template
# >>> should override torch.utils.Dataset

class CustomDataset(data.Dataset):
    
    def __init__(self, img_list, label_list, domain_list, transform=None):
        self.img_list = img_list
        self.label_list = label_list
        self.domain_list = domain_list
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        
        image = Image.open(self.img_list[idx])
        label = self.label_list[idx]
        domain = [self.domain_list[idx]]

        if self.transform:
            image = self.transform(image)

        return image, label, domain

dataset = DF_dataset(img_list, label_list, domain_list, transformer)    

In [ ]:
# 2-3. ImageFolder using torchvision
# >>> torchvision.datasets.dataset.ImageFolder

path = '/PATH/TO/DATASET/'
dataset = dataset.ImageFolder(root=path,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                           ]))

### 3. Data Split & Loader

In [ ]:
train_len = int(len(dataset) * 0.9)
test_len = len(dataset) - train_len

train_dataset, test_dataset = data.random_split(dataset, lengths=[train_len, test_len])

train_loader = data.DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE,
                              num_workers=4
                              )

test_loader = data.DataLoader(test_dataset, shuffle=True, batch_size=BATCH_SIZE,
                              num_workers=4
                             )

### 4. Model

In [ ]:
class My_model(nn.Module):
    def __init__(self, pretrained, num_classes):
        super(My_model, self).__init__()
        
        self.pretrained = pretrained
        self.fc = nn.Sequential(nn.Linear(73728, 2024),
                                nn.ReLU(),
                                nn.Linear(2024, num_classes)
                               )
    
    def forward(self, x):
        x = self.pretrained.extract_features(x)
        x = self.fc(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pretrained = resnet101_cbam(pretrained=True).to(device)

#my_extended_model = nn.DataParallel(my_extended_model)    # Multi GPU
model = My_model(pretrained=pretrained, num_classes = 2)
model.to(device)
model
print('load model complete')

### 5. Train & Test Phase

In [ ]:
num_epochs = 100
running_loss = 0.0
total_step = len(train_loader)
print("total_step %d" % total_step)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for i, (images, labels) in tqdm(enumerate(train_loader)):  
        images = images.type(torch.FloatTensor).to(device)
        labels = labels.type(torch.LongTensor).to(device)

        # Forward pass
        outputs = model(images)

        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if (i+1) % int(total_step/10) == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
                
    model.eval()
    test_loss=0
    with torch.no_grad():
        correct = 0
        total = 0
        
        for images, labels in val_loader:
            images = images.type(torch.FloatTensor).to(device)
            labels = labels.type(torch.LongTensor).to(device)
            
            outputs = model(images)
            prediction = outputs.argmax(1)
            
            temp = outputs.tolist()
            test_loss += criterion(outputs, labels)
            correct += len([index for index, (e1, e2) in enumerate(zip(prediction, labels.tolist())) if e1 == e2])

            test_loss /= len(val_loader.dataset)
            
        print('\nTest set:\n\tAverage loss: {:.4f}'.format(test_loss))
        print('\tAccuracy: {}/{} ({:.0f}%)\n'.format(correct,(len(val_loader) * val_loader.batch_size),100. * correct / (len(val_loader) * val_loader.batch_size)))
        
        name = "savefile/1106_Resnet_binary/model"+str(epoch+1)+".pt"
        with open(name, "wb") as f:
            torch.save({            'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()},
                f)